### Could use some work here to make the classes look good and efficient

In [13]:
from __future__  import annotations
from collections import Counter, defaultdict, namedtuple, deque
from itertools   import permutations, combinations, cycle, product, islice, chain
from functools   import lru_cache
from typing      import Dict, Tuple, Set, List, Iterator, Optional
from sys         import maxsize

import re
import ast
import operator

import numpy as np

In [14]:
def read_data(input: str, parser=str, sep='\n', testing=False) -> list:
    if testing:
        sections = input.split(sep)
    else:
        sections = open(input).read().split(sep)
    return [parser(section) for section in sections]

In [15]:
string = """L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL"""

In [16]:
test_ins = read_data(string, parser=lambda x: list(x), sep="\n", testing=True)
test_ins


[['L', '.', 'L', 'L', '.', 'L', 'L', '.', 'L', 'L'],
 ['L', 'L', 'L', 'L', 'L', 'L', 'L', '.', 'L', 'L'],
 ['L', '.', 'L', '.', 'L', '.', '.', 'L', '.', '.'],
 ['L', 'L', 'L', 'L', '.', 'L', 'L', '.', 'L', 'L'],
 ['L', '.', 'L', 'L', '.', 'L', 'L', '.', 'L', 'L'],
 ['L', '.', 'L', 'L', 'L', 'L', 'L', '.', 'L', 'L'],
 ['.', '.', 'L', '.', 'L', '.', '.', '.', '.', '.'],
 ['L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L'],
 ['L', '.', 'L', 'L', 'L', 'L', 'L', 'L', '.', 'L'],
 ['L', '.', 'L', 'L', 'L', 'L', 'L', '.', 'L', 'L']]

In [251]:
class Seats():
    def __init__(self, seats: List[List[str]]):
        self.curr_seats = seats
        self.offsets = set(product((-1, 0, 1), (-1, 0, 1))) - set([(0, 0),])
        self.num_rows = len(self.curr_seats)
        self.num_cols = len(self.curr_seats[0])
        self.occupied = '#'
        self.free = 'L'
        self.magic_num = 4

    def is_valid(self, idx: List[int, int]) -> bool:
        row, col = idx
        if row < 0 or row >= self.num_rows or col < 0 or col >= self.num_cols:
            return False
        return True

    def neighboring_seat_idx(self, current_idx: List[int, int]) -> List[List[int, int]]:
        row, col = current_idx
        return [[row + drow, col + dcol] for drow, dcol 
                in self.offsets
                if self.is_valid([row + drow, col + dcol])]

    def num_adj_occupied(self, current_idx: List[int, int]) -> int:
        neighboring_vals = [self.curr_seats[row][col] for row, col 
                            in self.neighboring_seat_idx(current_idx)]
        return Counter(neighboring_vals)[self.occupied]

    def next_val(self, current_idx: List[int, int]) -> int:
        row, col = current_idx
        current_val = self.curr_seats[row][col]
        num_occupied = self.num_adj_occupied(current_idx)
        if current_val == self.occupied and  num_occupied >= self.magic_num:
            return self.free
        elif current_val == self.free and num_occupied == 0:
            return self.occupied
        return current_val
    
    def next_seats(self):
        return [[self.next_val([row, col]) 
                      for col in range(self.num_cols)]
                     for row in range(self.num_rows)]

    def iterate(self):        
        new_seats = self.next_seats()
        while self.curr_seats != new_seats:
        # for i in range(3):
            # from pprint import pprint as pp
            # pp([''.join(row) for row in self.curr_seats])
            self.curr_seats = new_seats
            new_seats = self.next_seats()
    
    def count_occupied(self):
        self.iterate()
        return Counter(''.join([''.join(row) for row in self.curr_seats]))[self.occupied]
            

In [252]:
test_seats = Seats(test_ins)
test_seats.count_occupied()

37

Part I  

Simulate your seating area by applying the seating rules repeatedly until no seats change state. How many seats end up occupied?

In [253]:
%%timeit
real_ins = read_data("input.txt", parser=lambda x: list(x))
real_seats = Seats(real_ins)
real_seats.count_occupied()

4.12 s ± 5.77 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [254]:
real_seats.count_occupied()

2359

Part II

Given the new visibility method and the rule change for occupied seats becoming empty, once equilibrium is reached, how many seats end up occupied?

In [255]:
class NewSeats(Seats):
    def __init__(self, seats: List[List[str]]):
        super().__init__(seats)
        self.magic_num = 5

    def neighboring_seat_idx(self, current_idx: List[int, int]) -> List[List[int, int]]:
        row, col = current_idx
        idx = []
        for drow, dcol in self.offsets:
            ddrow = drow
            ddcol = dcol
            while self.is_valid([row + drow, col + dcol]) and not (self.curr_seats[row + drow][col + dcol] == self.occupied or self.curr_seats[row + drow][col + dcol] == self.free):
                        drow += ddrow
                        dcol += ddcol
            if self.is_valid([row + drow, col + dcol]):
                idx.append([row + drow, col + dcol])
        return idx
    

In [256]:
test_seats2 = NewSeats(test_ins)
test_seats2.offsets
test_seats2.count_occupied()

26

In [257]:
test1 = """.##.##.
#.#.#.#
##...##
...L...
##...##
#.#.#.#
.##.##."""
test_ins1 = read_data(test1, parser=lambda x: list(x), sep="\n", testing=True)
test_seats2 = NewSeats(test_ins1)
test_seats2.offsets
test_seats2.count_occupied()

9

In [258]:
real_seats2 = NewSeats(real_ins)
real_seats2.count_occupied()

2131